# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [2]:
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
import datetime
import cv2

GPU limitation

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-07-06 10:55:17.818344: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 10:55:17.955122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 10:55:17.955372: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-06 10:55:17.992199: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Building the Model

### Train

In [4]:
default_data = np.loadtxt('./datasets/model2_data/default.txt')
falling_data = np.loadtxt('./datasets/model2_data/falling.txt')

In [5]:
mu, sig = 0, 0.001
copy_count = len(default_data)-len(falling_data)
oldLen = len(falling_data)
rowi = 0

# limit = lambda x : max(min(x,1), 0)

def limit(x):
    if x > 1: return 1 - (x-1)
    elif x < 0: return abs(x)
    return x

newLimit = np.vectorize(limit)

for i in range(copy_count):
    err = np.random.normal(mu, sig, 16)
    scale = 1
    row = falling_data[rowi]*scale + err
    row = newLimit(row)

    falling_data = np.vstack([falling_data, row])
    rowi += 1
    if rowi >= oldLen: rowi = 0

print(falling_data.shape, default_data.shape)

(6270, 16) (6270, 16)


In [34]:
# loading data
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data)))) #ones are falling , zeros are default
dataset_size = len(inputs)
new_indices = np.random.permutation(dataset_size) # shuffle indices to shuffle X and y at the same time
inputs, outputs = inputs[new_indices], outputs[new_indices]

train_size = int(0.8*dataset_size)
test_size = dataset_size - train_size


# train_data = tf.data.Dataset.from_tensor_slices((inputs[:train_size], outputs[:train_size])).batch(128)
X_train = tf.data.Dataset.from_tensor_slices(inputs[:train_size]).batch(128)
y_train = tf.data.Dataset.from_tensor_slices(outputs[:train_size]).batch(128)
X_test, y_test = inputs[train_size:], outputs[train_size:]
print(X_test)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 1. 1.]]


In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [35]:
model.fit(x=X_train, y=y_train, epochs=6,callbacks=[tensorboard_callback])

Epoch 1/6
314/314 [==============================] - 2s 5ms/step - loss: 0.3758 - accuracy: 0.8552
Epoch 2/6
314/314 [==============================] - 2s 5ms/step - loss: 0.3751 - accuracy: 0.8564
Epoch 3/6
314/314 [==============================] - 2s 5ms/step - loss: 0.3736 - accuracy: 0.8566
Epoch 4/6
314/314 [==============================] - 2s 7ms/step - loss: 0.3744 - accuracy: 0.8576
Epoch 5/6
314/314 [==============================] - 2s 6ms/step - loss: 0.3723 - accuracy: 0.8586
Epoch 6/6
314/314 [==============================] - 2s 5ms/step - loss: 0.3715 - accuracy: 0.8583


### Evaluate & Export

In [30]:
loss, acc = model.evaluate(X_test, y_test)
# model.summary()
# model.save("model2")

40/40 [==============================] - 0s 991us/step - loss: 0.3782 - accuracy: 0.8485


In [8]:
### CONVERT TO TFLITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model2_tflite = converter.convert()
with open('model2.tflite', 'wb') as f: f.write(model2_tflite)

INFO:tensorflow:Assets written to: /tmp/tmpgy6fdty2/assets


2023-06-07 11:16:44.155686: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2023-06-07 11:16:44.445558: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-06-07 11:16:44.445580: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-06-07 11:16:44.446467: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpgy6fdty2
2023-06-07 11:16:44.447205: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-06-07 11:16:44.447233: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpgy6fdty2
2023-06-07 11:16:44.449735: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-06-07 11:16:44.489124: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on Saved

In [29]:
### QUANTIZE
def representative_dataset():
  for d in inputs:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('model2_quant', 'wb') as f: f.write(tflite_quant_model2)

INFO:tensorflow:Assets written to: /tmp/tmpdvgpu7f5/assets


INFO:tensorflow:Assets written to: /tmp/tmpdvgpu7f5/assets
/home/jessica/VIA-Fall-Detection/.venv/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-06-07 15:22:06.987031: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-06-07 15:22:06.987052: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-06-07 15:22:06.987225: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpdvgpu7f5
2023-06-07 15:22:06.987830: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-06-07 15:22:06.987841: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpdvgpu7f5
2023-06-07 15:22:06.989672: I tensorflow/cc/sav

# Predict

## Reformat Input

In [47]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [62]:
# Resize vid
video_path = "compilation_cut.mp4"
output_path = "compilation_cut_resized.mp4"
target_width = 224
target_height = 224

#resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

29.97


## Pass into model 1

In [63]:
# Initialize list to store the frame classifications
frame_classifications = []
moving_prob = []
# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    
    prob = np.around(max(output_probs.numpy()), decimals = 2)
    if predicted_class == "Still":
        
        moving_prob.append(np.subtract(1, prob))
    else:
        moving_prob.append(prob)

## Prepare data for model 2

In [64]:
moving_probs_trimmed = moving_prob[:-(len(moving_prob)%16)]
model2_in = np.array(moving_probs_trimmed).reshape((len(moving_prob)//16, 16))
print(model2_in)

[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   1.   1.   1.   1.   1.   1.   1.   1.   1.
  1.   1.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.   1.   1.   1.   1.   1.   1.
  1.   0.  ]
 [0.   0.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
  1.   1.  ]
 [1.   1.   1.   1.   1.   1.   1.   1.   1.   0.   0.   1.   0.   1.
  1.   1.  ]
 [1.   1.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.   1.   1.
  1.   1.  ]
 [1.   1.   1.   1.   1.   0.   0.   1.   1.   1.   0.   1.   1.   0.
  1.   1.  ]
 [0.   0.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
  1.   1.  ]
 [1.

## Pass into Model 2 and get final prediction

In [65]:
model2 = tf.keras.models.load_model("model2")

In [66]:
vid_preds = model2.predict(model2_in)

In [67]:
threshold = 0.88
bools = vid_preds.reshape((1, len(vid_preds))) > threshold
print(bools)
print(vid_preds.reshape((1, len(vid_preds))))

[[False False False False False False False False False False False False
   True  True False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False  True False  True  True False
  False False False False False False False False False False False False
  False False]]
[[0.09045138 0.09045138 0.09045138 0.09045138 0.09045138 0.5739342
  0.30349308 0.7925048  0.7975426  0.51072216 0.7444853  0.7925048
  0.929349   0.929349   0.8644172  0.09045138 0.15479052 0.29671496
  0.09045138 0.15544447 0.09045138 0.09045138 0.09045138 0.09045138
  0.09045138 0.17014171 0.12607063 0.09045138 0.09045138 0.19399568
  0.09045138 0.09045138 0.16418059 0.14767478 0.15647146 0.12583217
  0.09045138 0.09045138 0.18011312 0.09045138 0.11525431 0.17111975
  0.26666683 0.929349   0.80626893 0.929349   0.929349   0.2670536
  0.16497837 0.09045138 0.09045138 0.09045138 0.09045138 0.09045138
  0.0